<h1><center>Capstone Project: Segmenting and Clustering Neighborhoods in Toronto</center></h1>
<h3><center>Fetch Table data from Wikipedia</center></h3>
<h4>About this Notebook</h4>
<br>created by <a href="https://github.com/Sherryl93/">@Sherryl93</a>

<br>
<br> Data Source: <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M</a>

In [1]:
#Libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [62]:
# fetch data from wikipedia and parse the HTML/XML 
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')


In [63]:
# extracting the raw table inside that webpage
table = soup.find('table')

## Extract data from Raw Table

In [188]:
#Declare empty list
postal_list   = []
borough_list  = []
neighbor_list = []


# begin loop to fetch data 
for tr_cell in table.find_all('tr'):
    for td_cell in tr_cell.find_all('td'):
        #print(td_cell)
        # loop trhough postal:
        for b in td_cell.find_all('b'):
            postal_list.append(b.string)
        if not(td_cell.find_all('i') or td_cell.find_all('a')):
            str1,str2 = td_cell.find('span',{'style':'font-size:80%;'}).text.replace(')', '').split('(')
            borough_list.append(str1)
            neighbor_list.append(str2)
        elif td_cell.find_all('i'):
            for i in td_cell.find_all('i'):
                borough_list.append(i.string)
            neighbor_list.append(None)
        else:
            counter = 0
            temp_neigh  = [] #for temp only
            for a in td_cell.find_all('a'):
                #print(a.get('title'))
                if counter == 0:
                    borough_list.append(a.string)
                    #print(a.string)
                    counter+=1
                else:
                    temp_neigh.append(a.string)
            if len(temp_neigh)>1:
                neighbor_list.append(','.join(temp_neigh))
            else:
                try:
                    neighbor_list.append(temp_neigh[0])
                except:
                    neighbor_list.append(None)
            

# check len of each list
print("Number of PostalCode:", len(postal_list))
print("Number of Borough:", len(borough_list))
print("Number of Neighborhood:", len(neighbor_list))

Number of PostalCode: 180
Number of Borough: 180
Number of Neighborhood: 180


## Integrate Data

In [190]:
data = {'PostalCode': postal_list, 'Borough': borough_list, 'Neighborhood':neighbor_list}
df = pd.DataFrame.from_dict(data)
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
5,M6A,North York,"Lawrence Manor,Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
7,M8A,Not assigned,None
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern,Rouge"


In [196]:
#Drop column if column contained not assigned
df_cleaned = df[(df.Borough != 'Not assigned')& (df.Borough.notna()) & (df.Neighborhood.notna())]
df_cleaned

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park,Harbourfront"
5,M6A,North York,"Lawrence Manor,Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern,Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill,Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District,Ryerson"


In [199]:
df_cleaned.shape

(90, 3)